In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

df = pd.read_csv("Datos/pokemon.csv")
df2 = pd.read_csv("Datos/combats.csv")

# En este caso el modelo se aplica a categorías que no responden a valores numéricos continuos.
# Se utiliza de forma binaria, para predecir si un cliente va a comprar un determinado producto o no.
# Los valores devueltos por la expresión se encuentran entre 0 y 1.

In [3]:
# Vamos a determinar el retraso en la llegada a partir del retraso en la salida.
# Ambas variables están muy correlacionadas.
df = df.dropna(subset = ["height_m"])
df = df.sample(frac = 1).head(100000)
Y = df["height_m"] < 1 # Esta será nuestra variable respuesta: vuelos sin y con retraso (True = 1 / False = 0)
X = df[["weight_kg"]]

In [4]:
# Creamos el modelo con LogisticRegression()
logreg = LogisticRegression()
# Ajustamos el modelo con fit
logreg.fit(X, Y)
# Obtenemos las predicciones
Y_pred = logreg.predict(X)

In [5]:
# La regresión logística nos devuelve probabilidades, selecciona la categoría con probabilidad superior.
np.round(logreg.predict_proba(X),3)
# El primer caso tiene una probabilidad de un 1% de pertenecer al grupo 0 (sin retraso) y 
# un 98.2 de pertenecer al grupo 1 (con retraso)

array([[0.154, 0.846],
       [0.527, 0.473],
       [1.   , 0.   ],
       ...,
       [1.   , 0.   ],
       [0.169, 0.831],
       [0.996, 0.004]])

In [6]:
# 95,59% de las predicciones es correcta. Vemos en qué caso nuestras predicciones coinciden con el valor real.
np.mean(Y_pred == Y)

0.8412291933418694